In [1]:
import BioSimSpace as bss
import glob
import functions as fn
from BioSimSpace import _Exceptions

INFO:rdkit:Enabling RDKit 2022.09.1 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [6]:
config = "/home/jguven/projects/metalloenzymes/scripts/test.cfg"
minimisation_config = ["ncycles = 1\n", "minimise = True\n", "minimise maximum iterations = 10000\n"]
with open(config, "r") as config:
    lines = config.readlines()

for i in range(len(lines)):
    for j in range(len(minimisation_config)):
        key = minimisation_config[j].split()[0]
        if key in lines[i]:
            print(lines[i])

ncycles = 5

ncycles_per_snap = 1



In [42]:
unbound_path = "../kpc2/equilibration/unbound/"
bound_path = "../kpc2/equilibration/bound/"

system_1 = bss.IO.readMolecules([f"{bound_path}/system_1/npt/system_1.prm7",
                                        f"{bound_path}/system_1/npt/system_1.rst7"])

system_2 = bss.IO.readMolecules([f"{bound_path}/system_2/npt/system_2.prm7",
                                        f"{bound_path}/system_2/npt/system_2.rst7"])


In [43]:
system_1_ligand = None
protein = None
n_residues = [molecule.nResidues() for molecule in system_1]
n_atoms = [molecule.nAtoms() for molecule in system_1]
for j, (n_residues, n_atoms) in enumerate(zip(n_residues[:20], n_atoms[:20])):
    if n_residues == 1 and n_atoms > 5:
        system_1_ligand = system_1.getMolecule(j)
    elif n_residues > 1:
        protein = system_1.getMolecule(j)
    else:
        pass

system_2_ligand = None
n_residues = [molecule.nResidues() for molecule in system_2]
n_atoms = [molecule.nAtoms() for molecule in system_2]
for j, (n_residues, n_atoms) in enumerate(zip(n_residues, n_atoms)):
    if n_residues == 1 and n_atoms > 5:
        system_2_ligand = system_2.getMolecule(j)
    else:
        pass    

if system_1_ligand and system_2_ligand and protein:
    print(f"got molecules")
else:
    raise _Exceptions.AlignmentError("Could not extract ligands or protein from input systems.")

mapping = bss.Align.matchAtoms(system_1_ligand, system_2_ligand, complete_rings_only=True)
inverse_mapping = {value:key for key,value in mapping.items()}

system_2_ligand_aligned = bss.Align.rmsdAlign(system_2_ligand, system_1_ligand, inverse_mapping)

bound_merged_ligands = bss.Align.merge(system_1_ligand, system_2_ligand_aligned, mapping)

system_1.removeMolecules(system_1_ligand)
system_1.addMolecules(bound_merged_ligands)
bound_system = system_1


got molecules


In [47]:
lambda_string = "0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0"
lambda_list = lambda_string.split()
lambdas = [float(value) for value in lambda_list]
print(lambdas)

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]


In [52]:

free_energy_protocol = bss.Protocol.FreeEnergy(lam_vals=lambdas, runtime=5*bss.Units.Time.nanosecond)

# working_directory = f"{full_path}/{system_name}/outputs/{engines[i].strip()}/lig_1~lig_2"
bss.FreeEnergy.Relative(bound_system, free_energy_protocol, engine="GROMACS", work_dir= "bound/", setup_only=True)



In [11]:
ligand_path = "../kpc2/inputs/ligands/"
solvated_ligand_files = glob.glob(ligand_path + f"ligand_1_solvated.*")

print(solvated_ligand_files)
test_work_dir = "test_min/"
minimisation_steps = 250

['../kpc2/inputs/ligands/ligand_1_solvated.top', '../kpc2/inputs/ligands/ligand_1_solvated.gro']


In [74]:
solvated_ligand = bss.IO.readMolecules(solvated_ligand_files)

ligand_minimisation_protocol = bss.Protocol.Minimisation(steps=minimisation_steps)
ligand_minimisation_process = bss.Process.Gromacs(solvated_ligand,
                                                  ligand_minimisation_protocol,
                                                  name="min",
                                                  work_dir=test_work_dir)

# fn.edit_mdp_options(test_work_dir, "min", {"emstep": 0.001, "emtol": 1000})
ligand_minimisation_process.setConfig(["emstep = 0.001", "emtol = 2000"])
ligand_minimisation_process.setArg("-ntmpi", 1)


In [75]:
ligand_minimisation_process.getConfig()
ligand_minimisation_process.getArgs()

OrderedDict([('mdrun', True), ('-deffnm', 'min'), ('-ntmpi', 1)])

In [76]:
ligand_minimisation_process.start()

# Wait for the process to exit.
ligand_minimisation_process.wait()

# Check for errors.
if ligand_minimisation_process.isError():
    print(ligand_minimisation_process.stdout())
    print(ligand_minimisation_process.stderr())
    raise _Exceptions.ThirdPartyError("The process exited with an error!")

# If it worked, try to get the system. No need to block, since it's already finished.
min_system = ligand_minimisation_process.getSystem()

In [77]:
ligand_minimisation_process.stdout()

NOTE: 85 % of the run time was spent in pair search,
      you might want to increase nstlist (this has no effect on accuracy)

               Core t (s)   Wall t (s)        (%)
       Time:        0.215        0.007     3152.9
                 (ns/day)    (hour/ns)
Performance:       12.687        1.892

GROMACS reminds you: "What do you call an acid with an attitude? A-mean-oh acid." (Anonymous)



In [22]:
unbound_path = "../kpc2/equilibration/unbound/"
bound_path = "../kpc2/equilibration/bound/"


ligand_1_system_gmx = bss.IO.readMolecules([f"{unbound_path}/ligand_1/npt/npt.gro",
                                            f"{unbound_path}/ligand_1/npt/npt.top"])
ligand_2_system_gmx = bss.IO.readMolecules([f"{unbound_path}/ligand_2/npt/npt.gro",
                                            f"{unbound_path}/ligand_2/npt/npt.top"])



In [7]:
ligand_1_system_gmx.getMolecules()[0].isPerturbable()

False